In [ ]:
import os
from pathlib import Path
from typing import Callable

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
from sqlalchemy import create_engine

In [ ]:
DATABASE_URL = os.environ["DB_URL_RPC"]

In [ ]:
OUTPUT_PATH = Path("outputs")

In [ ]:
db_engine = create_engine(DATABASE_URL)

# Requêtes de construction des cohortes


Table pour les cohortes CEE

```sql
create table luis.cee_drivers as (select
	i."uuid",
	min(ca."_id"::text) as id_first_cee,
	min(c.datetime) as date_first_cee,
	min(ca.operator_id) as id_operateur_cee
from carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner join cee.cee_applications ca on
	c."_id" = ca.carpool_id
where ca.journey_type = 'short'
and not ca.is_specific
group by 1)
```

---

```sql
create table luis.cee_drivers_v2 as (
with cee_trip as (
	select
		ca."_id"::text as id_first_cee,
		c.identity_id as identity_id_cee,
		c.datetime as date_first_cee,
		ca.operator_id as id_operateur_cee
	from cee.cee_applications ca
	inner join carpool.carpools c  on c._id = ca.carpool_id
	where ca.journey_type = 'short'
	and not ca.is_specific
)
select
	i."uuid",
	ct.*,
	o."name" as nom_operateur
from cee_trip ct inner join carpool.identities i on ct.identity_id_cee=i._id
inner join "operator".operators o on o._id = ct.id_operateur_cee
)

```

---

```sql
create table luis.cee_drivers_v3 as (
with trips as (select
	i."uuid",
	c.trip_id,
	c.datetime
from carpool.carpools c
inner join carpool.identities i on c.identity_id = i."_id"
 where is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
 ),
cee_trip as (
	select
		ca."_id"::text as id_first_cee,
		c.identity_id as identity_id_cee,
		c.datetime as date_first_cee,
		ca.operator_id as id_operateur_cee
	from cee.cee_applications ca
	inner join carpool.carpools c  on c._id = ca.carpool_id
	where ca.journey_type = 'short'
	and not ca.is_specific
)
select
	i."uuid",
	max(ct.id_first_cee) as id_first_cee,
	max(ct.date_first_cee) as date_first_cee,
	case
		when max(ct.date_first_cee) between '2023-01-01' and '2023-03-31' then 't1_23'
		when max(ct.date_first_cee) between '2023-04-01' and '2023-06-30' then 't2_23'
		when max(ct.date_first_cee) between '2023-07-01' and '2023-09-30' then 't3_23'
		when max(ct.date_first_cee) between '2023-10-01' and '2023-12-31' then 't4_23'
	end as cohorte,
	max(o."name") as nom_operateur,
	count(distinct ft.trip_id) filter (where date_first_cee > ft.datetime) as num_trips_before_cee,
	min(ft.datetime) as date_first_trip
from cee_trip ct inner join carpool.identities i on ct.identity_id_cee=i._id
inner join "operator".operators o on o._id = ct.id_operateur_cee
left join trips ft on i."uuid" = ft.uuid
group by 1
)
```


Table pour la cohorte 2022

```sql
create table luis.cohorte_2022 as (
	select
	  i.uuid,
		min(c.datetime) AS date_first_trip
	from
		carpool.carpools c
    left join carpool.identities i on c.identity_id = i._id
    where
    is_driver
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
    group by
      1
    HAVING min(c.datetime) BETWEEN '2022-01-01' AND '2022-09-30'
)
```


# Statistiques CEE globales


In [ ]:
df_cee_by_operator_month = pl.read_database(
    """
select 
	date_trunc('month',datetime) as "month",
	operator_id,
	max(o.name) as operator_name,
	count(*) as num_cee
from cee.cee_applications ca 
left join "operator".operators o on ca.operator_id = o."_id" 
where journey_type = 'short'
and not is_specific 
group by 1,2
order by 1,2
""",
    connection=db_engine,
)

In [ ]:
color_mapping = {
    k: v
    for k, v in zip(
        df_cee_by_operator_month.filter(pl.col("operator_name") != "Picholines")[
            "operator_name"
        ]
        .unique()
        .sort()
        .to_list(),
        [
            "#f8c291",
            "#82ccdd",
            "#78e08f",
            "#e55039",
            "#f6b93b",
            "#4a69bd",
            "#3c6382",
            "#78e08f",
            "#0c2461",
            "#b71540",
            "#3d3d3d",
            "#cd84f1",
        ],
    )
}

In [ ]:
fig_cee_by_operator_and_month = px.bar(
    df_cee_by_operator_month,
    x="month",
    y="num_cee",
    color="operator_name",
    color_discrete_map=color_mapping,
    template="simple_white",
    labels={"month": "Mois", "num_cee": "Nombre de CEE"},
    title="Evolution du nombre de CEE par mois et par opérateurs",
)
fig_cee_by_operator_and_month.show()
fig_cee_by_operator_and_month.write_html(OUTPUT_PATH / "fig_stats_cee_mensuelles.html")
fig_cee_by_operator_and_month.write_image(
    OUTPUT_PATH / "fig_stats_cee_mensuelles.svg", format="svg", width=1280, height=720
)

# Statistique cohortes CEE


In [ ]:
df_cohortes_cee = pl.read_database(
    """
select
	*
from
	luis.cee_drivers_v3
where cohorte is not null
""",
    connection=db_engine,
)

In [ ]:
df_cohortes_cee.describe()

In [ ]:
df_cohortes_cee_by_operator = (
    df_cohortes_cee.group_by(["cohorte", "nom_operateur"])
    .agg(pl.col("uuid").len())
    .with_columns(
        (100 * pl.col("uuid") / pl.col("uuid").sum().over("cohorte"))
        .round(2)
        .alias("share")
    )
    .sort(["cohorte", "nom_operateur"], descending=[False, True])
)

In [ ]:
traces = []
for operator in sorted(df_cohortes_cee_by_operator["nom_operateur"].unique().to_list()):
    data = df_cohortes_cee_by_operator.filter(pl.col("nom_operateur") == operator).sort(
        "cohorte"
    )
    trace = go.Bar(
        x=data["cohorte"].replace(
            {
                "t1_23": "CEE T1 2023",
                "t2_23": "CEE T2 2023",
                "t3_23": "CEE T3 2023",
                "t4_23": "CEE T4 2023",
            }
        ),
        y=data["share"],
        text=data["share"],
        texttemplate="%{text:.2f}%",
        hovertemplate="%{fullData.name} représente %{text:.2f}% des conducteurs de la cohorte %{x}<extra></extra>",
        name=operator,
        marker_color=color_mapping[operator],
    )
    traces.append(trace)
fig_cohortes_stats = go.Figure(traces)
fig_cohortes_stats.update_layout(barmode="stack", plot_bgcolor="white")
fig_cohortes_stats.update_yaxes(
    range=[0, 115],
    showgrid=True,
    gridcolor="black",
    griddash="dot",
    layer="above traces",
    showline=True,
    title="Part des conducteurs",
)
fig_cohortes_stats.update_xaxes(
    zeroline=True, zerolinecolor="black", zerolinewidth=0.5, title="Cohorte"
)
fig_cohortes_stats.show()

In [ ]:
fig_cohortes_stats.write_html(OUTPUT_PATH / "stats_cohortes_cee.html")
fig_cohortes_stats.write_image(
    OUTPUT_PATH / "stats_cohortes_cee.svg", format="svg", width=1280, height=720
)

# Nombre de trajets effectués par an


## Requêtes


In [ ]:
df_trips_by_driver = pl.read_database(
    """
select
	i.uuid,
	count(distinct trip_id) as num_trajets,
    min(c.datetime) as date_premier_trajet,
    max(c.datetime) as date_dernier_trajet
from
	carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
WHERE c.datetime BETWEEN ft.date_first_trip AND ft.date_first_trip + INTERVAL '12 weeks'
and is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
""",
    connection=db_engine,
)

In [ ]:
df_trips_by_driver.describe()

In [ ]:
df_trips_by_driver_cohortes = pl.read_database(
    """
with cohortes as (select 
	*
from luis.cee_drivers_v3 cd)
select 
	ch.uuid,
	count(distinct trip_id) as num_trajets,
	min(c.datetime) as date_premier_trajet,
	max(c.datetime) as date_dernier_trajet,
	max(ch.date_first_cee) as date_premier_cee,
	max(ch.cohorte) as cohorte,
  max(ch.nom_operateur) as nom_operateur
from
	carpool.carpools c
inner join carpool.identities i on
	c.identity_id = i._id
inner join cohortes ch on
	i."uuid" = ch.uuid
where
	c.datetime BETWEEN ch.date_first_cee AND ch.date_first_cee+ INTERVAL '12 weeks'
	and ch.cohorte is not null
	and c.is_driver
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by 1
""",
    connection=db_engine,
)

In [ ]:
df_trips_by_driver_cohortes.describe()

In [ ]:
df_trips_by_driver_cohortes.group_by(pl.col("cohorte")).agg(
    pl.count().alias("nombre_de_conducteurs")
).sort(pl.col("cohorte").str.split("_").list.reverse().list.join(""))

In [ ]:
df_trips_by_driver_cohortes.group_by("cohorte").agg(
    pl.col("num_trajets").mean().alias("moyenne_nombre_trajets")
).sort(pl.col("cohorte").str.split("_").list.reverse().list.join(""))

## Comparaison 2022 vs CEE


In [ ]:
def preprocess_trips_by_driver_df(df: pl.DataFrame, bins: list[int]) -> pl.DataFrame:
    return (
        df.get_column("num_trajets")
        .hist(bins, include_breakpoint=True)
        .with_columns(
            pl.col("break_point")
            .cast(pl.String)
            .replace(np.inf, f"{bins[-1]+1}+")
            .str.replace("(\.0)", ""),
            (100 * pl.col("count") / pl.col("count").sum()).alias("share"),
        )
    )

In [ ]:
bins = list(range(49))
plot_configs = [
    {
        "data": preprocess_trips_by_driver_df(df_trips_by_driver, bins),
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": preprocess_trips_by_driver_df(
            df_trips_by_driver_cohortes.filter(pl.col("cohorte") == "t1_23"), bins
        ),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": preprocess_trips_by_driver_df(
            df_trips_by_driver_cohortes.filter(pl.col("cohorte") == "t2_23"), bins
        ),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": preprocess_trips_by_driver_df(
            df_trips_by_driver_cohortes.filter(pl.col("cohorte") == "t3_23"), bins
        ),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": preprocess_trips_by_driver_df(
            df_trips_by_driver_cohortes.filter(pl.col("cohorte") == "t4_23"), bins
        ),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": preprocess_trips_by_driver_df(
            df_trips_by_driver_cohortes.filter(pl.col("cohorte") == "t1_24"), bins
        ),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        x=data["break_point"],
        y=data["share"],
        marker_color=config["color"],
        name=config["name"],
        hovertemplate="%{y:.2f}% des conducteurs ont fait %{x} trajets",
    )
    traces.append(trace)

fig_trips_by_drivers_multi = go.Figure(traces)
fig_trips_by_drivers_multi.update_layout(
    barmode="group",
    bargroupgap=0.2,
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Distribution du nombre de trajets effectués pour chaque cohorte",
)
fig_trips_by_drivers_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des conducteurs (%)",
)
fig_trips_by_drivers_multi.update_xaxes(title="Nombre de trajets effectués")
fig_trips_by_drivers_multi.add_vrect(
    x0=9.5,
    x1=10.5,
    fillcolor="#7f8c8d",
    opacity=0.25,
    line_width=0,
    annotation_text="BONUS CEE",
    annotation_position="top left",
    annotation_textangle=-90,
    annotation_font_size=10,
)
fig_trips_by_drivers_multi.show()
fig_trips_by_drivers_multi.write_html(
    OUTPUT_PATH / "histo_trajets_par_conducteurs_multi.html"
)
fig_trips_by_drivers_multi.write_image(
    OUTPUT_PATH / "histo_trajets_par_conducteurs_multi.svg",
    format="svg",
    width=1280,
    height=720,
)

## Comparaison par opérateur


In [ ]:
bins = list(range(48))


ref_data = preprocess_trips_by_driver_df(df_trips_by_driver, bins)
traces = [
    go.Bar(
        x=ref_data["break_point"],
        y=ref_data["share"],
        name="Cohorte 2022",
        hovertemplate="%{y:.2f}% des conducteurs ont fait %{x} trajets",
        marker_color="#f39c12",
        marker_pattern_shape="x",
        marker_pattern_size=12,
        marker_pattern_fgcolor="black",
        marker_pattern_fgopacity=1,
    )
]

enabled_traces = ["YNSTANT", "BlaBlaCar Daily", "MOOVANCE", "Klaxit"]
for operateur in df_trips_by_driver_cohortes["nom_operateur"].unique():
    data = preprocess_trips_by_driver_df(
        df_trips_by_driver_cohortes.filter(pl.col("nom_operateur") == operateur), bins
    )
    trace = go.Bar(
        x=data["break_point"],
        y=data["share"],
        text=operateur,
        name=operateur,
        hovertemplate="%{y:.2f}% des conducteurs ont fait %{x} trajets",
        marker_color=color_mapping[operateur],
        visible=True if operateur in enabled_traces else "legendonly",
    )
    traces.append(trace)

fig_trips_by_drivers_multi_op = go.Figure(traces)
fig_trips_by_drivers_multi_op.update_layout(
    barmode="group",
    bargroupgap=0.2,
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Distribution du nombre de trajets effectués pour chaque cohorte",
    template="seaborn",
)
fig_trips_by_drivers_multi_op.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des conducteurs (%)",
)
fig_trips_by_drivers_multi_op.update_xaxes(
    title="Nombre de trajets effectués", range=[0.5, 10.5]
)
fig_trips_by_drivers_multi_op.add_vrect(
    x0=9.5,
    x1=10.5,
    fillcolor="#7f8c8d",
    opacity=0.25,
    line_width=0,
    annotation_text="BONUS CEE",
    annotation_position="top left",
    annotation_textangle=-90,
    annotation_font_size=10,
)
fig_trips_by_drivers_multi_op.show()
fig_trips_by_drivers_multi_op.write_html(
    OUTPUT_PATH / "histo_trajets_par_conducteurs_multi_op.html"
)
fig_trips_by_drivers_multi_op.write_image(
    OUTPUT_PATH / "histo_trajets_par_conducteurs_multi_op.svg",
    format="svg",
    width=1280,
    height=720,
)

# Nombre de semaines d'activité


## Requêtes


In [ ]:
df_activity_weeks_by_driver = pl.read_database(
    """
select
	i.uuid,
	count(distinct trip_id) as num_trajets,
	count(distinct date_trunc('week',c.datetime)) as num_semaines_activité
from
	carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
WHERE c.datetime BETWEEN ft.date_first_trip AND ft.date_first_trip + INTERVAL '12 WEEKS'
and is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
""",
    connection=db_engine,
)

In [ ]:
df_activity_weeks_by_driver.describe()

In [ ]:
df_activity_weeks_by_driver_cohortes = pl.read_database(
    """
with cohortes as (select 
	*
from luis.cee_drivers_v3 cd)
select 
	ch.uuid,
	count(distinct trip_id) as num_trajets,
	count(distinct date_trunc('week',c.datetime)) as num_semaines_activité,
    min(c.datetime) as date_premier_trajet,
    max(c.datetime) as date_dernier_trajet,
	max(ch.date_first_cee) as date_premier_cee,
	max(ch.cohorte) as cohorte
from
	carpool.carpools c
inner join carpool.identities i on
	c.identity_id = i._id
inner join cohortes ch on
	i."uuid" = ch.uuid
where
	c.datetime BETWEEN ch.date_first_cee AND ch.date_first_cee+ INTERVAL '12 WEEKS'
    and ch.cohorte is not null
	and c.is_driver
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by 1
""",
    connection=db_engine,
)

In [ ]:
df_activity_weeks_by_driver_cohortes.describe()

In [ ]:
df_activity_weeks_by_driver_cohortes.group_by("cohorte").agg(
    pl.col("num_semaines_activité").mean().alias("moyenne_nombre_semaines_activité")
).sort(pl.col("cohorte").str.split("_").list.reverse().list.join(""))

## Comparaison 2022 vs 2023 CEE


In [ ]:
def preprocess_activity_week_by_driver_df(
    df: pl.DataFrame, bins: list[int]
) -> pl.DataFrame:
    return (
        df.get_column("num_semaines_activité")
        .hist(bins, include_breakpoint=True)
        .with_columns(
            pl.col("break_point")
            .cast(pl.String)
            .replace(np.inf, f"{bins[-1]+1}")
            .str.replace("(\.0)", ""),
            (100 * pl.col("count") / pl.col("count").sum()).alias("share"),
        )
    )

In [ ]:
bins = list(range(13))
plot_configs = [
    {
        "data": preprocess_activity_week_by_driver_df(
            df_activity_weeks_by_driver, bins
        ),
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": preprocess_activity_week_by_driver_df(
            df_activity_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t1_23"),
            bins,
        ),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": preprocess_activity_week_by_driver_df(
            df_activity_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t2_23"),
            bins,
        ),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": preprocess_activity_week_by_driver_df(
            df_activity_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t3_23"),
            bins,
        ),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": preprocess_activity_week_by_driver_df(
            df_activity_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t4_23"),
            bins,
        ),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": preprocess_activity_week_by_driver_df(
            df_activity_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t1_24"),
            bins,
        ),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        x=data["break_point"],
        y=data["share"],
        marker_color=config["color"],
        name=config["name"],
        hovertemplate="%{y:.2f}% des conducteurs ont %{x} semaine(s) d'activité",
    )
    traces.append(trace)

fig_activity_weeks_drivers_multi = go.Figure(traces)
fig_activity_weeks_drivers_multi.update_layout(
    barmode="group",
    bargroupgap=0.2,
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Nombre de semaines d'activité par conducteur<br>Une semaine d'activité est définie comme une semaine où le conducteur a effectué au moins un trajet",
)
fig_activity_weeks_drivers_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des conducteurs (%)",
)
fig_activity_weeks_drivers_multi.update_xaxes(title="Nombre de semaines d'activité")
fig_activity_weeks_drivers_multi.show()
fig_activity_weeks_drivers_multi.write_html(
    OUTPUT_PATH / "histo_semaines_activites_par_conducteurs_multi.html"
)
fig_activity_weeks_drivers_multi.write_image(
    OUTPUT_PATH / "histo_semaines_activites_par_conducteurs_multi.svg",
    format="svg",
    width=1280,
    height=720,
)

# Distance


## Requêtes


In [ ]:
df_distance_by_trips = pl.read_database(
    """
select
	trip_id,
	max(distance) as distance
from
	carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
WHERE c.datetime BETWEEN ft.date_first_trip AND ft.date_first_trip + INTERVAL '12 weeks'
and is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
""",
    connection=db_engine,
)

In [ ]:
df_distance_by_trips.describe()

In [ ]:
df_distance_by_trips_cohortes = pl.read_database(
    """
with cohortes as (select 
	*
from luis.cee_drivers_v3 cd)
select
	trip_id,
	max(distance) as distance,
    max(ft.cohorte) as cohorte
from
	carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner JOIN cohortes ft ON ft.uuid=i.uuid
WHERE c.datetime BETWEEN ft.date_first_cee AND ft.date_first_cee + INTERVAL '12 weeks'
and is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
and ft.cohorte is not null
group by
	1
""",
    connection=db_engine,
)

In [ ]:
df_distance_by_trips_cohortes.describe()

In [ ]:
df_distance_by_trips_cohortes.group_by("cohorte").agg(pl.col("trip_id").count()).sort(
    "cohorte"
).sort(pl.col("cohorte").str.split("_").list.reverse().list.join(""))

In [ ]:
df_distance_by_trips_cohortes.group_by("cohorte").agg(
    (pl.col("distance") / 1000).mean().alias("moyenne_distance")
).sort(pl.col("cohorte").str.split("_").list.reverse().list.join(""))

## Comparaison 2022 vs 2023 CEE


In [ ]:
plot_configs = [
    {
        "data": df_distance_by_trips,
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": df_distance_by_trips_cohortes.filter(pl.col("cohorte") == "t1_23"),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": df_distance_by_trips_cohortes.filter(pl.col("cohorte") == "t2_23"),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": df_distance_by_trips_cohortes.filter(pl.col("cohorte") == "t3_23"),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": df_distance_by_trips_cohortes.filter(pl.col("cohorte") == "t4_23"),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": df_distance_by_trips_cohortes.filter(pl.col("cohorte") == "t1_24"),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Histogram(
        x=data["distance"] / 1000,
        histfunc="count",
        histnorm="percent",
        xbins_size=0.2,
        xbins_start=0,
        marker_color=config["color"],
        marker_opacity=0.5,
        name=config["name"],
        hovertemplate="%{y:.2f}% des trajets font %{x} km",
        visible=(
            True
            if config["name"] in ["Référence 2022", "CEE T4 2023", "CEE T1 2024"]
            else "legendonly"
        ),
    )
    traces.append(trace)

fig_distance_multi = go.Figure(traces)
fig_distance_multi.update_layout(
    barmode="overlay",
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Distribution des distances réalisées pour les trajets de chaque cohorte",
)
fig_distance_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des trajets (%)",
)
fig_distance_multi.update_xaxes(title="Distance réalisée (km)", range=[0, 70])
fig_distance_multi.show()
fig_distance_multi.write_html(OUTPUT_PATH / "histo_distances_multi.html")
fig_distance_multi.write_image(
    OUTPUT_PATH / "histo_distances_multi.svg", format="svg", width=1920, height=1080
)

# Heures de départ


## Requêtes


In [ ]:
df_departure_dow_hour_by_trips = pl.read_database(
    """
SELECT 
	trip_id,
	date_part('dow',min(datetime at time zone 'Europe/Paris')) as jour,
	date_part('hour',min(datetime at time zone 'Europe/Paris')) as heure
from
	carpool.carpools c
    left join carpool.identities i on c.identity_id = i._id
    inner JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
    WHERE c.datetime BETWEEN ft.date_first_trip AND ft.date_first_trip + INTERVAL '12 WEEKS'
    and is_driver
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
 """,
    connection=db_engine,
)

In [ ]:
df_departure_dow_hour_by_trips.describe()

In [ ]:
df_departure_dow_hour_by_trips_cohortes = pl.read_database(
    """
with cohortes as (select 
	*,
	case 
		when cd.date_first_cee between '2023-01-01' and '2023-03-31' then 't1_23'
		when cd.date_first_cee between '2023-04-01' and '2023-06-30' then 't2_23'
		when cd.date_first_cee between '2023-07-01' and '2023-09-30' then 't3_23'
		when cd.date_first_cee between '2023-10-01' and '2023-12-31' then 't4_23'
	end as cohorte
from luis.cee_drivers_v2 cd)
SELECT 
	trip_id,
	date_part('dow',min(datetime at time zone 'Europe/Paris')) as jour,
	date_part('hour',min(datetime at time zone 'Europe/Paris')) as heure,
    max(ft.cohorte) as cohorte
from
	carpool.carpools c
    left join carpool.identities i on c.identity_id = i._id
    inner JOIN cohortes ft ON ft.uuid=i.uuid
    WHERE c.datetime BETWEEN ft.date_first_cee AND ft.date_first_cee + INTERVAL '12 WEEKS'
    and ft.uuid is not null
    and is_driver
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
""",
    connection=db_engine,
)

In [ ]:
df_departure_dow_hour_by_trips_cohortes.describe()

## Comparaison


In [ ]:
def preprocess_departure_dow_hour_by_trips_df(
    departure_dow_hour_by_trips_df: pl.DataFrame,
) -> pl.DataFrame:
    mapping_jours = {
        0: "Dimanche",
        6: "Samedi",
        5: "Vendredi",
        4: "Jeudi",
        3: "Mercredi",
        2: "Mardi",
        1: "Lundi",
    }
    departure_dow_hour_by_trips_df_agg = (
        departure_dow_hour_by_trips_df.with_columns(
            pl.col("jour").cast(pl.Int8), pl.col("heure").cast(pl.Int8)
        )
        .group_by(["jour", "heure"])
        .agg(
            (
                100
                * pl.col("trip_id").count()
                / departure_dow_hour_by_trips_df.shape[0]
            ).alias("share")
        )
        .with_columns(pl.col("jour").replace(mapping_jours).alias("jour_str"))
    ).sort((pl.col("jour") - 1) % 7, descending=True)

    return departure_dow_hour_by_trips_df_agg

In [ ]:
data = preprocess_departure_dow_hour_by_trips_df(df_departure_dow_hour_by_trips)
trace = go.Heatmap(
    x=data["heure"],
    y=data["jour_str"],
    z=data["share"],
    xgap=2,
    ygap=2,
    autocolorscale=False,
    colorscale=px.colors.sequential.Peach,
    hovertemplate="%{z:.2f}% des trajets ont lieu le jour %{y} de %{x}H00 à %{x}H59<extra></extra>",
    colorbar_title="Part des trajets",
    colorbar_ticksuffix="%",
)
fig_dow_hours_trips = go.Figure([trace])
fig_dow_hours_trips.update_layout(
    plot_bgcolor="white", title="Cohorte 2022 - Quand ont lieu les départs de trajets ?"
)
fig_dow_hours_trips.update_xaxes(dtick=1, title="Heure de la journée")
fig_dow_hours_trips.update_yaxes(dtick=1, title="Jour de la semaine")
fig_dow_hours_trips.show()

In [ ]:
data = preprocess_departure_dow_hour_by_trips_df(
    df_departure_dow_hour_by_trips_cohortes.filter(pl.col("cohorte") == "t4_23")
)
trace = go.Heatmap(
    x=data["heure"],
    y=data["jour_str"],
    z=data["share"],
    xgap=2,
    ygap=2,
    autocolorscale=False,
    colorscale=px.colors.sequential.Peach,
    hovertemplate="%{z:.2f}% des trajets ont lieu le jour %{y} de %{x}H00 à %{x}H59<extra></extra>",
    colorbar_title="Part des trajets",
    colorbar_ticksuffix="%",
)
fig_dow_hours_trips_cee = go.Figure([trace])
fig_dow_hours_trips_cee.update_layout(
    plot_bgcolor="white",
    title="Cohorte CEE T4 23 - Quand  ont lieu les départs de trajets ?",
)
fig_dow_hours_trips_cee.update_xaxes(dtick=1, title="Heure de la journée")
fig_dow_hours_trips_cee.update_yaxes(dtick=1, title="Jour de la semaine")
fig_dow_hours_trips_cee.show()

# Passagers


## Requêtes


In [ ]:
df_passengers_by_trips = pl.read_database(
    """
SELECT 
	c.trip_id,
    max(c.datetime) as date_trajet,
    max(ft.date_first_trip) as date_first_trip,
	sum(seats) as "count",
	max(operator_id) as operator_id,
    max(ft.uuid::text) as uuid_cohorte
from
	carpool.carpools c
    left join carpool.identities i on c.identity_id = i._id
    left JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
Having
   max(c.datetime)  BETWEEN  max(ft.date_first_trip) AND  max(ft.date_first_trip) + INTERVAL '12 weeks'
   and max(ft.uuid::text) is not null
""",
    connection=db_engine,
)

In [ ]:
df_passengers_by_trips.describe()

In [ ]:
df_passengers_by_trips_cohortes = pl.read_database(
    """
with cohortes as (select 
	*
from luis.cee_drivers_v3 cd)
SELECT 
	c.trip_id,
    max(c.datetime) as date_trajet,
    max(ft.date_first_cee) as date_first_trip,
	sum(seats) as "count",
	max(operator_id) as operator_id,
    max(ft.uuid::text) as uuid_cohorte,
    max(ft.cohorte) as cohorte
from
	carpool.carpools c
    left join carpool.identities i on c.identity_id = i._id
    left JOIN cohortes ft ON ft.uuid=i.uuid
    and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
Having
   max(c.datetime)  BETWEEN  max(ft.date_first_cee) AND  max(ft.date_first_cee) + INTERVAL '12 weeks'
   and max(ft.uuid::text) is not null
   and max(ft.cohorte) is not null
""",
    connection=db_engine,
)

In [ ]:
df_passengers_by_trips_cohortes.describe()

In [ ]:
df_passengers_by_trips_cohortes.group_by("cohorte").agg(pl.col("count").mean()).sort(
    pl.col("cohorte").str.split("_").list.reverse().list.join("")
)

## Visualisation


In [ ]:
def preprocess_passengers_by_trips_df(
    df: pl.DataFrame, bins: list[int]
) -> pl.DataFrame:
    return (
        df.get_column("count")
        .hist(bins, include_breakpoint=True)
        .with_columns(
            pl.col("break_point")
            .cast(pl.String)
            .replace(np.inf, f"{bins[-1]+1}")
            .str.replace("(\.0)", ""),
            (100 * pl.col("count") / pl.col("count").sum()).alias("share"),
        )
    )

In [ ]:
bins = list(range(5))
plot_configs = [
    {
        "data": preprocess_passengers_by_trips_df(df_passengers_by_trips, bins),
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": preprocess_passengers_by_trips_df(
            df_passengers_by_trips_cohortes.filter(pl.col("cohorte") == "t1_23"),
            bins,
        ),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": preprocess_passengers_by_trips_df(
            df_passengers_by_trips_cohortes.filter(pl.col("cohorte") == "t2_23"),
            bins,
        ),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": preprocess_passengers_by_trips_df(
            df_passengers_by_trips_cohortes.filter(pl.col("cohorte") == "t3_23"),
            bins,
        ),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": preprocess_passengers_by_trips_df(
            df_passengers_by_trips_cohortes.filter(pl.col("cohorte") == "t4_23"),
            bins,
        ),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": preprocess_passengers_by_trips_df(
            df_passengers_by_trips_cohortes.filter(pl.col("cohorte") == "t1_24"),
            bins,
        ),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        x=data["break_point"],
        y=data["share"],
        marker_color=config["color"],
        name=config["name"],
        hovertemplate="%{y:.2f}% des trajets ont %{x} passager(s)",
    )
    traces.append(trace)

fig_passagers_multi = go.Figure(traces)
fig_passagers_multi.update_layout(
    barmode="group",
    bargroupgap=0.2,
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Combien y-a t'il de passagers dans un trajet de coviturage ?",
)
fig_passagers_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des trajets (%)",
)
fig_passagers_multi.update_xaxes(title="Nombre de passagers", range=[0.5, 5.5])
fig_passagers_multi.show()
fig_passagers_multi.write_html(OUTPUT_PATH / "histo_passagers_multi.html")
fig_passagers_multi.write_image(
    OUTPUT_PATH / "histo_passagers_multi.svg", format="svg", width=1280, height=720
)

# Trajets intracommunaux


## Requêtes


In [ ]:
df_communes_by_trips = pl.read_database(
    """
select
	trip_id,
	min(start_geo_code) as start_commune,
	min(end_geo_code) as end_commune
from
	carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
WHERE c.datetime BETWEEN ft.date_first_trip AND ft.date_first_trip + INTERVAL '12 weeks'
and is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	1
""",
    connection=db_engine,
)

In [ ]:
df_communes_by_trips.describe()

In [ ]:
df_communes_by_trips.select(
    (pl.col("start_commune") == pl.col("end_commune")).value_counts()
).unnest("start_commune").with_columns(100 * pl.col("count") / pl.col("count").sum())

In [ ]:
df_communes_by_trips_cohortes = pl.read_database(
    """
with cohortes as (select 
	*
from luis.cee_drivers_v3 cd)
select
	trip_id,
	min(start_geo_code) as start_commune,
	min(end_geo_code) as end_commune,
    max(ft.cohorte) as cohorte
from
	carpool.carpools c
inner join carpool.identities i on c.identity_id = i._id
inner JOIN cohortes ft ON ft.uuid=i.uuid
WHERE c.datetime BETWEEN ft.date_first_cee AND ft.date_first_cee + INTERVAL '12 weeks'
and is_driver
and status=cast('ok' as covoiturage_production.carpool.carpool_status_enum)
and ft.cohorte is not null
group by
	1
""",
    connection=db_engine,
)

## Comparaison


In [ ]:
def preprocess_communes_by_trips(df: pl.DataFrame) -> float:
    truth_share = (
        100
        * (
            df.select((pl.col("start_commune") == pl.col("end_commune")).value_counts())
            .unnest("start_commune")
            .filter(pl.col("start_commune"))
            .select("count")
            / len(df)
        ).item()
    )

    return truth_share

In [ ]:
plot_configs = [
    {
        "data": preprocess_communes_by_trips(df_communes_by_trips),
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": preprocess_communes_by_trips(
            df_communes_by_trips_cohortes.filter(pl.col("cohorte") == "t1_23")
        ),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": preprocess_communes_by_trips(
            df_communes_by_trips_cohortes.filter(pl.col("cohorte") == "t2_23")
        ),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": preprocess_communes_by_trips(
            df_communes_by_trips_cohortes.filter(pl.col("cohorte") == "t3_23")
        ),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": preprocess_communes_by_trips(
            df_communes_by_trips_cohortes.filter(pl.col("cohorte") == "t4_23")
        ),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": preprocess_communes_by_trips(
            df_communes_by_trips_cohortes.filter(pl.col("cohorte") == "t1_24")
        ),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        x=[config["name"]],
        y=[data],
        text=[f"{data:.2f}%"],
        textposition="inside",
        textfont_size=15,
        marker_color=config["color"],
        hovertemplate="%{y:.2f}% des trajets sont intra-communaux<extra></extra>",
    )
    traces.append(trace)

fig_communes_multi = go.Figure(traces)
fig_communes_multi.update_layout(
    barmode="overlay",
    plot_bgcolor="white",
    showlegend=False,
    title="Part des trajets dont le départ et la destination est la même commune",
    margin_t=80,
)
fig_communes_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des trajets intracommunaux (%)",
)
fig_communes_multi.update_xaxes(title="Cohorte")
fig_communes_multi.show()
fig_communes_multi.write_html(OUTPUT_PATH / "histo_communes_multi.html")
fig_communes_multi.write_image(OUTPUT_PATH / "histo_communes_multi.svg", format="svg")

# Churn/Utilisation/Attrition


## Requêtes


In [ ]:
df_weeks_by_driver = pl.read_database(
    """
with "template" as (
select 
	*,
	generate_series(date_trunc('week',c.date_first_trip at time zone 'Europe/Paris'),
	date_trunc('week',c.date_first_trip at time zone 'Europe/Paris' + interval '19 weeks'),
	interval '1 weeks') as semaine
from
	luis.cohorte_2022 c
  where c.date_first_trip < '2022-08-01'
  ),

trips as (
select
	ft.uuid,
	date_trunc('week',
	c.datetime at time zone 'Europe/Paris') as semaine
from
	carpool.carpools c
inner join carpool.identities i on
c.identity_id = i._id
inner JOIN luis.cohorte_2022 ft ON ft.uuid=i.uuid
where
c.datetime at time zone 'Europe/Paris' between ft.date_first_trip and ft.date_first_trip + interval '19 weeks'
and is_driver
and status = cast('ok' as covoiturage_production.carpool.carpool_status_enum)
group by
	 1,2),

aggregated_data as (select 
	t.uuid::text,
	t.semaine,
	count(tr.semaine)>0 had_trip
from
	"template" t
left join trips tr on
	t.uuid = tr.uuid
	and t.semaine = tr.semaine
group by
	1,2)

select 
	*,
	row_number() over (partition by uuid order by semaine) as num_semaine
from aggregated_data
order by 1,2
""",
    connection=db_engine,
)

In [ ]:
df_weeks_by_driver.describe()

In [ ]:
df_weeks_by_driver_cohortes = pl.read_database(
    """
with "template" as (
select
	*,
  generate_series(date_trunc('week',cd.date_first_cee at time zone 'Europe/Paris'),
	                date_trunc('week',cd.date_first_cee at time zone 'Europe/Paris' + interval '19 weeks'),
                  interval '1 weeks') as semaine
from luis.cee_drivers_v3 cd
  ),

trips as (
select
	ft.uuid,
	date_trunc('week',
	c.datetime) as semaine
from
	carpool.carpools c
inner join carpool.identities i on
c.identity_id = i._id
inner JOIN "template" ft ON ft.uuid=i.uuid
where
c.datetime between ft.date_first_cee and ft.date_first_cee + interval '19 weeks'
and is_driver
and status = cast('ok' as covoiturage_production.carpool.carpool_status_enum)
and ft.cohorte is not null
group by
	 1,2),

aggregated_data as (select 
	t.uuid::text,
	t.semaine,
	count(tr.semaine)>0 had_trip,
  max(t.cohorte) as cohorte
from
	"template" t
left join trips tr on
	t.uuid = tr.uuid
	and t.semaine = tr.semaine
group by
	1,2)

select 
	*,
	row_number() over (partition by uuid order by semaine) as num_semaine
from aggregated_data
order by 1,2
""",
    connection=db_engine,
)

In [ ]:
df_weeks_by_driver_cohortes.describe()

In [ ]:
df_weeks_by_driver_cohortes["cohorte"].unique()

## Comparaison


In [ ]:
def preprocess_week_by_driver_df(df: pl.DataFrame) -> pl.DataFrame:
    df_week_by_cohorte = (
        df.group_by(["num_semaine"])
        .agg((100 * pl.col("had_trip").sum() / df["uuid"].n_unique()).alias("share"))
        .sort("num_semaine")
    )
    return df_week_by_cohorte

In [ ]:
plot_configs = [
    {
        "data": preprocess_week_by_driver_df(df_weeks_by_driver),
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": preprocess_week_by_driver_df(
            df_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t1_23")
        ),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": preprocess_week_by_driver_df(
            df_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t2_23")
        ),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": preprocess_week_by_driver_df(
            df_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t3_23")
        ),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": preprocess_week_by_driver_df(
            df_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t4_23")
        ),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": preprocess_week_by_driver_df(
            df_weeks_by_driver_cohortes.filter(pl.col("cohorte") == "t1_24")
        ),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Scatter(
        x=data["num_semaine"],
        y=data["share"],
        marker_color=config["color"],
        hovertemplate="%{y:.2f}% des conducteurs ont été actifs %{x} semaine(s)",
        name=config["name"],
        visible=config.get("visible", True),
    )
    traces.append(trace)

fig_weeks_by_driver_multi = go.Figure(traces)
fig_weeks_by_driver_multi.update_layout(
    plot_bgcolor="white",
    title="Courbes d'attrition des conducteurs<br>Quel cohorte a réussi à retenir les conducteurs le plus longtemps ?",
    hovermode="x unified",
)
fig_weeks_by_driver_multi.update_yaxes(
    showgrid=True,
    griddash="dot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des conducteurs (%)",
    zeroline=True,
    zerolinecolor="black",
    tickvals=[0, 10, 20, 40, 60, 80, 100],
)
fig_weeks_by_driver_multi.update_xaxes(
    title="Numéro de la semaine",
    range=[0, 21],
    dtick=1,
    tickprefix="Semaine ",
    showtickprefix="none",
)

fig_weeks_by_driver_multi.add_annotation(
    x=19,
    y=10,
    text="Stabilisation autour de 7%",
    showarrow=True,
    arrowhead=1,
    font_color="rgba(44, 62, 80,1.0)",
    bgcolor="white",
    font_size=14,
    borderpad=5,
)

# Annotation du coude
fig_weeks_by_driver_multi.add_annotation(
    x=2.5,
    y=58.5,
    ay=63,
    ayref="y",
    ax=3,
    axref="x",
    text="Entre 50% et 60% des utilisateurs perdus en semaine 2",
    xanchor="left",
    showarrow=True,
    arrowhead=1,
    font_color="rgba(44, 62, 80,1.0)",
    bgcolor="white",
    font_size=14,
    borderpad=3,
)
fig_weeks_by_driver_multi.add_shape(
    type="circle",
    xref="x",
    yref="y",
    x0=1.5,
    y0=32,
    x1=2.5,
    y1=60,
    line_dash="dash",
)
fig_weeks_by_driver_multi.show()
fig_weeks_by_driver_multi.write_html(OUTPUT_PATH / "num_semaines_multi.html")
fig_weeks_by_driver_multi.write_image(
    OUTPUT_PATH / "num_semaines_multi.svg", format="svg", width=1920, height=1080
)

# Primo-conducteurs


In [ ]:
df_primo_drivers = pl.read_database(
    """
select
  *
from luis.cee_Drivers_v3 cdv 
where cdv.cohorte is not null
  """,
    connection=db_engine,
)

In [ ]:
df_primo_drivers.group_by("num_trips_before_cee").len().sort("num_trips_before_cee")

## Visualisation


In [ ]:
def preprocess_trips_pre_cee_by_driver_df(
    df: pl.DataFrame, bins: list[int]
) -> pl.DataFrame:
    df_agg = (
        df.get_column("num_trips_before_cee")
        .hist(bins, include_breakpoint=True)
        .with_columns(
            pl.col("break_point")
            .cast(pl.String)
            .replace(np.inf, f"{bins[-1]+1}+")
            .str.replace("(\.0)", ""),
            (100 * pl.col("count") / pl.col("count").sum()).alias("share"),
        )
    )
    return df_agg

In [ ]:
bins = list(range(6))
plot_configs = [
    {
        "data": preprocess_trips_pre_cee_by_driver_df(
            df_primo_drivers.filter(pl.col("cohorte") == "t1_23"), bins
        ),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": preprocess_trips_pre_cee_by_driver_df(
            df_primo_drivers.filter(pl.col("cohorte") == "t2_23"), bins
        ),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": preprocess_trips_pre_cee_by_driver_df(
            df_primo_drivers.filter(pl.col("cohorte") == "t3_23"), bins
        ),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": preprocess_trips_pre_cee_by_driver_df(
            df_primo_drivers.filter(pl.col("cohorte") == "t4_23"), bins
        ),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": preprocess_trips_pre_cee_by_driver_df(
            df_primo_drivers.filter(pl.col("cohorte") == "t4_23"), bins
        ),
        "name": "CEE T4 2023",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        x=data["break_point"],
        y=data["share"],
        marker_color=config["color"],
        hovertemplate="%{y:.2f}% des conducteurs ont été actifs %{x} semaine(s)",
        name=config["name"],
    )
    traces.append(trace)

fig_weeks_by_driver_multi = go.Figure(traces)
fig_weeks_by_driver_multi.update_layout(
    plot_bgcolor="white",
    barmode="group",
    title="Nombre de trajets effectués pré-CEE",
)
fig_weeks_by_driver_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des conducteurs (%)",
)
fig_weeks_by_driver_multi.update_xaxes(title="Nombre de trajets effectués avant la CEE")
fig_weeks_by_driver_multi.show()
fig_weeks_by_driver_multi.write_html(OUTPUT_PATH / "num_trajets_pre_cee_multi.html")
fig_weeks_by_driver_multi.write_image(
    OUTPUT_PATH / "num_trajets_pre_cee_multi.svg", format="svg", width=1280, height=720
)

# Distance économisée


## Requêtes


In [ ]:
# La distance est multipliée par le nombre de sièges
df_distance_eco_by_driver = pl.read_database(
    """
with drivers_trips as (
select 
		i.uuid,
		trip_id
from
	carpool.carpools c
inner join carpool.identities i on
	c.identity_id = i._id
inner join luis.cohorte_2022 cdv on
	i."uuid" = cdv."uuid"
where
	c.datetime between cdv.date_first_trip and cdv.date_first_trip + interval '12 weeks'
	and status = cast('ok' as covoiturage_production.carpool.carpool_status_enum)
	and is_driver
group by
		1,
	2
),
trips_distance as (
select 
	c.trip_id,
	sum(coalesce (distance,(c.meta->>'calc_distance')::int) * seats) as distance_passagers
from
	carpool.carpools c
where
	c.trip_id in (
	select
		trip_id
	from
		drivers_trips)
group by
	1)
select 
	a.uuid,
	sum(b.distance_passagers)::float as distance
from
	drivers_trips a
left join trips_distance b on
	a.trip_id = b.trip_id
group by
	a.uuid
""",
    connection=db_engine,
)

In [ ]:
df_distance_eco_by_driver_cee = pl.read_database(
    """
with drivers_trips as (
select 
		i.uuid,
		trip_id,
    max(cdv.cohorte) as cohorte,
    max(cdv.nom_operateur) as nom_operateur
from
	carpool.carpools c
inner join carpool.identities i on
	c.identity_id = i._id
inner join luis.cee_drivers_v3 cdv on
	i."uuid" = cdv."uuid"
where
	c.datetime between cdv.date_first_cee and cdv.date_first_cee + interval '12 weeks'
	and cdv.cohorte is not null
	and status = cast('ok' as covoiturage_production.carpool.carpool_status_enum)
	and is_driver
group by
		1,
	2
),
trips_distance as (
select 
	c.trip_id,
	sum(coalesce (distance,(c.meta->>'calc_distance')::int) * seats) as distance_passagers
from
	carpool.carpools c
where
	c.trip_id in (
	select
		trip_id
	from
		drivers_trips)
group by
	1)
select 
	a.uuid,
  max(a.cohorte) as cohorte,
	sum(b.distance_passagers)::float as distance
from
	drivers_trips a
left join trips_distance b on
	a.trip_id = b.trip_id
group by
	a.uuid
""",
    connection=db_engine,
)

## Stats


In [ ]:
df_distance_eco_by_driver.describe()

In [ ]:
df_distance_eco_by_driver.select(pl.col("distance").sum())

In [ ]:
df_distance_eco_by_driver_cee.describe()

In [ ]:
df_distance_eco_by_driver_cee.select(pl.col("distance").sum() / 1000)

## Visualisation


In [ ]:
plot_configs = [
    {
        "data": df_distance_eco_by_driver,
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t1_23"),
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t2_23"),
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t3_23"),
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t4_23"),
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t1_24"),
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Histogram(
        x=data["distance"] / 1000,
        histfunc="count",
        histnorm="percent",
        xbins_size=1,
        xbins_start=0,
        marker_color=config["color"],
        marker_opacity=0.5,
        name=config["name"],
        hovertemplate="%{y:.2f}% des conducteurs ont économisés %{x} km",
        visible=(
            True
            if config["name"] in ["Référence 2022", "CEE T4 2023", "CEE T1 2024"]
            else "legendonly"
        ),
    )
    traces.append(trace)

fig_distance_eco_multi = go.Figure(traces)
fig_distance_eco_multi.update_layout(
    barmode="overlay",
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Distribution des distances économisés par les conducteurs de chaque cohorte",
)
fig_distance_eco_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Part des conducteurs (%)",
)
fig_distance_eco_multi.update_xaxes(title="Distance économisée (km)", range=[0, 70])
fig_distance_eco_multi.show()
fig_distance_eco_multi.write_html(OUTPUT_PATH / "histo_distances_eco_multi.html")
fig_distance_eco_multi.write_image(
    OUTPUT_PATH / "histo_distances_eco_multi.svg", format="svg", width=1280, height=720
)

In [ ]:
df_distance_eco_by_driver["distance"].mean() / 1000

In [ ]:
plot_configs = [
    {
        "data": df_distance_eco_by_driver["distance"].mean() / 1000,
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t1_23")[
            "distance"
        ].mean()
        / 1000,
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t2_23")[
            "distance"
        ].mean()
        / 1000,
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t3_23")[
            "distance"
        ].mean()
        / 1000,
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t4_23")[
            "distance"
        ].mean()
        / 1000,
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t1_24")[
            "distance"
        ].mean()
        / 1000,
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        y=[data],
        x=[config["name"]],
        text=[f"{data:.2f}km"],
        marker_color=config["color"],
        name=config["name"],
        hovertemplate="Les conducteurs de la cohorte %{x} ont économisés en moyenne %{y}km",
    )
    traces.append(trace)

fig_distance_eco_moyenne_multi = go.Figure(traces)
fig_distance_eco_moyenne_multi.update_layout(
    barmode="overlay",
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Quelle distance un conducteur de chaque cohorte économise-t-il en moyenne ?",
)
fig_distance_eco_moyenne_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Distance moyenne <br>économisée par conducteur (km)",
)
fig_distance_eco_moyenne_multi.update_xaxes(title="Cohorte")
fig_distance_eco_moyenne_multi.show()
fig_distance_eco_moyenne_multi.write_html(
    OUTPUT_PATH / "distances_eco_moyenne_multi.html"
)
fig_distance_eco_moyenne_multi.write_image(
    OUTPUT_PATH / "distances_eco_moyenne_multi.svg",
    format="svg",
    width=1280,
    height=720,
)

In [ ]:
plot_configs = [
    {
        "data": df_distance_eco_by_driver["distance"].median() / 1000,
        "name": "Référence 2022",
        "color": "#f39c12",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t1_23")[
            "distance"
        ].median()
        / 1000,
        "name": "CEE T1 2023",
        "color": "#d7e1ed",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t2_23")[
            "distance"
        ].median()
        / 1000,
        "name": "CEE T2 2023",
        "color": "#89a6c7",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t3_23")[
            "distance"
        ].median()
        / 1000,
        "name": "CEE T3 2023",
        "color": "#3E6DA1",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t4_23")[
            "distance"
        ].median()
        / 1000,
        "name": "CEE T4 2023",
        "color": "#1a334e",
    },
    {
        "data": df_distance_eco_by_driver_cee.filter(pl.col("cohorte") == "t1_24")[
            "distance"
        ].median()
        / 1000,
        "name": "CEE T1 2024",
        "color": "rgba(113, 88, 226,1.0)",
    },
]
traces = []
for config in plot_configs:
    data = config["data"]
    trace = go.Bar(
        y=[data],
        x=[config["name"]],
        text=[f"{data:.2f}km"],
        marker_color=config["color"],
        name=config["name"],
        hovertemplate="Les conducteurs de la cohorte %{x} ont économisés une médiane de %{y}km",
    )
    traces.append(trace)

fig_distance_eco_mediane_multi = go.Figure(traces)
fig_distance_eco_mediane_multi.update_layout(
    barmode="overlay",
    plot_bgcolor="white",
    legend_title="Cohorte :",
    title="Quelle distance médiane un conducteur de chaque cohorte économise-t-il ?",
)
fig_distance_eco_mediane_multi.update_yaxes(
    showgrid=True,
    griddash="dashdot",
    gridwidth=1,
    gridcolor="gray",
    title="Distance médiane <br>économisée par conducteur (km)",
)
fig_distance_eco_mediane_multi.update_xaxes(title="Cohorte")
fig_distance_eco_mediane_multi.show()
fig_distance_eco_mediane_multi.write_html(
    OUTPUT_PATH / "distances_eco_mediane_multi.html"
)
fig_distance_eco_mediane_multi.write_image(
    OUTPUT_PATH / "distances_eco_mediane_multi.svg",
    format="svg",
    width=1280,
    height=720,
)